In [6]:
import pandas as pd
customers = pd.read_csv('olist_customers_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
names = pd.read_csv('product_category_name_translation.csv')

In [7]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [8]:

"""df_Vol['IBM'] = df_IBM['Volume']
df_Vol['MSoft'] = df_MSFT['Volume']
df_Vol['Google'] = df_GOOGL['Volume']

df_Vol.head()"""

"df_Vol['IBM'] = df_IBM['Volume']\ndf_Vol['MSoft'] = df_MSFT['Volume']\ndf_Vol['Google'] = df_GOOGL['Volume']\n\ndf_Vol.head()"

In [19]:
customers[['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']].shape

(99441, 4)

In [22]:
items[['order_id', 'product_id', 'price']].shape

(112650, 3)

In [24]:
payments[['order_id', 'payment_type']].shape

(103886, 2)

In [32]:
orders[['order_id','customer_id','order_purchase_timestamp']].shape

(99441, 3)

In [31]:
products[['product_id', 'product_category_name']].shape

(32951, 2)

In [38]:
names.shape

(71, 2)

In [42]:
product_names = pd.DataFrame()
product_names = pd.merge(products[['product_id', 'product_category_name']],names, on="product_category_name")
product_names[['product_id', 'product_category_name_english']].shape

(32328, 2)

In [49]:
items_and_names = pd.DataFrame()
items_and_names = pd.merge(items[['order_id', 'product_id', 'price']],product_names[['product_id', 'product_category_name_english']], on="product_id")
items_names_payments = pd.DataFrame()
items_names_payments = pd.merge(items_and_names,payments[['order_id', 'payment_type']], on="order_id")
items_names_payments_orders = pd.DataFrame()
items_names_payments_orders = pd.merge(items_names_payments,orders[['order_id','customer_id','order_purchase_timestamp']], on="order_id")
all = pd.DataFrame()
all = pd.merge(items_names_payments_orders,customers[['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']], on="customer_id")
all.head()

,order_id,product_id,price,product_category_name_english,payment_type,customer_id,order_purchase_timestamp,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,credit_card,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,28013,campos dos goytacazes,RJ
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.9,cool_stuff,boleto,e6eecc5a77de221464d1c4eaff0a9b64,2017-06-28 11:52:20,75800,jatai,GO
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.9,cool_stuff,credit_card,4ef55bf80f711b372afebcb7c715344a,2018-05-18 10:25:53,30720,belo horizonte,MG
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,credit_card,30407a72ad8b3f4df4d15369126b20c9,2017-08-01 18:38:42,83070,sao jose dos pinhais,PR
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.9,cool_stuff,credit_card,91a792fef70ecd8cc69d3c7feb3d12da,2017-08-10 21:48:40,36400,conselheiro lafaiete,MG


In [50]:
all.shape

(115878, 10)

In [59]:
len(all['order_id'].unique())

97255

In [78]:
all = all.drop_duplicates(subset='order_id',keep='first')

In [79]:
all.shape

(97255, 10)